In [ ]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
img_width = 64
img_height = 64

In [ ]:
tfrecord_train = 'train.tfrecord'
tfrecord_test = 'test.tfrecord'
tfrecord_dir = 'tfrecords'
cur_dir = os.getcwd()

In [ ]:
# hyper parameters
learning_rate = 0.001
training_epochs = 30
batch_size = 20
n_class = 20
n_train = 800
n_test = 200
seed = 777
tf.set_random_seed(seed)

In [ ]:
train_tfr_path = os.path.join(cur_dir, tfrecord_dir, tfrecord_train)
test_tfr_path = os.path.join(cur_dir, tfrecord_dir, tfrecord_test)

In [ ]:
def _parse_function(tfrecord_serialized):
    features={'image': tf.FixedLenFeature([], tf.string),
             'label': tf.FixedLenFeature([], tf.int64)}
    parsed_features = tf.parse_single_example(tfrecord_serialized, features)
    
    image = tf.decode_raw(parsed_features['image'], tf.uint8)
    label = tf.cast(parsed_features['label'], tf.int32)
    label_onehot = tf.one_hot(label, depth=n_class)
    
    print(image.shape)
    print(label_onehot.shape)
        
    #image = tf.reshape(image, [-1, img_height, img_width, 3])
    
    return image, label_onehot

In [ ]:
train_dataset = tf.data.TFRecordDataset(train_tfr_path)
train_dataset = train_dataset.map(_parse_function, num_parallel_calls=8)
train_dataset = train_dataset.shuffle(buffer_size=10000).prefetch(buffer_size=batch_size).batch(batch_size).repeat()
#print(train_dataset.output_types, train_dataset.output_shapes)

In [ ]:
test_dataset = tf.data.TFRecordDataset(test_tfr_path)
test_dataset = test_dataset.map(_parse_function, num_parallel_calls=8)
test_dataset = test_dataset.shuffle(buffer_size=10000).prefetch(buffer_size=batch_size).batch(batch_size).repeat()
#print(test_dataset.output_types, test_dataset.output_shapes)

In [ ]:
iterator = tf.data.Iterator.from_structure(train_dataset.output_types, train_dataset.output_shapes)
image, label = iterator.get_next()

In [ ]:
train_init = iterator.make_initializer(train_dataset)
test_init = iterator.make_initializer(test_dataset)

In [ ]:
image = tf.reshape(image, [-1, img_height, img_width, 3])
image = tf.cast(image, tf.float32) / 255.
is_train = tf.placeholder(tf.bool)

In [ ]:
conv1 = tf.layers.conv2d(inputs=image, filters=32, kernel_size=[3, 3],
                        padding="SAME", activation=tf.nn.relu)
pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2],
                        padding="SAME", strides=2)
bn1 = tf.layers.batch_normalization(pool1, training=is_train)
#bn1 = tf.contrib.layers.batch_norm(pool1, decay=0.9, is_training=is_train)
#drop1 = tf.layers.dropout(inputs=pool1, rate=0.3, training=is_train)

conv2 = tf.layers.conv2d(inputs=bn1, filters=64, kernel_size=[3, 3],
                        padding="SAME", activation=tf.nn.relu)
pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2],
                        padding="SAME", strides=2)
bn2 = tf.layers.batch_normalization(pool2, training=is_train)
#bn2 = tf.contrib.layers.batch_norm(pool2, decay=0.9, is_training=is_train)
#drop2 = tf.layers.dropout(inputs=pool2, rate=0.3, training=is_train)

conv3 = tf.layers.conv2d(inputs=bn2, filters=128, kernel_size=[3, 3],
                        padding="SAME", activation=tf.nn.relu)
pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2],
                        padding="SAME", strides=2)
bn3 = tf.layers.batch_normalization(pool3, training=is_train)
#bn3 = tf.contrib.layers.batch_norm(pool3, decay=0.9, is_training=is_train)
#drop3 = tf.layers.dropout(inputs=pool3, rate=0.3, training=is_train)

In [ ]:
flat3 = tf.contrib.layers.flatten(bn3)
dense4 = tf.layers.dense(inputs=flat3, units=625, activation=tf.nn.relu)
bn4 = tf.layers.batch_normalization(dense4, training=is_train)
#bn4 = tf.contrib.layers.batch_norm(dense4, decay=0.9, is_training=is_train)
#drop4 = tf.layers.dropout(inputs=dense4, rate=0.5, training=is_train)

logits = tf.layers.dense(inputs=bn4, units=n_class)

In [ ]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
    logits=logits, labels=label))
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
#optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [ ]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(label, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
# initialize
sess = tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth =True)))
sess.run(tf.global_variables_initializer())

In [ ]:
# train my model
print('Learning started. It takes sometime.')
max_test_acc = 0.
for epoch in range(training_epochs):
    avg_cost = 0.
    avg_train_acc = 0.
    avg_test_acc = 0.
    
    total_batch = int(n_train / batch_size)
    total_batch_test = int(n_test / batch_size)
    
    sess.run(train_init)

    for i in range(total_batch):
        #batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        #batch_xs = batch_xs.reshape(-1, time_steps, element_size)
        sess.run([image, label])        
        feed_dict = {is_train:True}
        acc, c, _ = sess.run([accuracy, cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch
        avg_train_acc += acc / total_batch
        
    sess.run(test_init)
        
    for i in range(total_batch_test):
        #batch_xs, batch_ys = mnist.test.next_batch(batch_size)        
        #batch_xs = batch_xs.reshape(-1, time_steps, element_size)
        sess.run([image, label])
        feed_dict = {is_train:False}
        acc = sess.run(accuracy, feed_dict=feed_dict)
        avg_test_acc += acc / total_batch_test

    print('Epoch:', '{}'.format(epoch + 1), 'cost =', '{:.9f}'.format(avg_cost), 
          'train accuracy = ', '{:.5f}'.format(avg_train_acc), 
          'test accuracy = ', '{:.5f}'.format(avg_test_acc))


print('Learning Finished!')